#Comparing the error rates for Monte Carlo and Riemann Integration
A commonly mentioned advantage of Monte Carlo integration is that the error rate does not depend on the dimensionality of the input space; it only depends on the number of samples used to estimate the integral. Let $\int_{\mathcal{H}} f(x)$ be the integral we want to estimate. In Monte Carlo integration, we use samples drawn uniformly from $\mathcal{H}$ to form our estimate as follows
$$F_{mc} = \frac{1}{N}(\sum_i f(x_i))$$
If we let $F$ to denote the true value of the integral, by the law of large numbers, $F_{mc}$ approaches $F$. Moreover, the variance of our estimate is $\frac{\sigma^2}{N}$ where $\sigma^2$ is the variance of $f(x)$. Hence, the error of Monte Carlo estimate varies with $O(n^{-1/2})$.

In Riemann integration, we simply partition $\mathcal{H}$ into $N$ equally spaced intervals and estimate the integral simply by
$$F_{r} = \frac{1}{N} \sum_i f(x_i) |H|$$
where $|H|$ is the size of the domain of integration. The error of Riemann estimate varies with $O(n^{-1})$, i.e., a better rate compared to Monte Carlo integration.

Let us first see that this is the case with a simple example. Let $f(x) = (1 - ||x||_2^2)$ where $x \in \mathbb{R}^D$. We use Monte Carlo and Riemann integration to estimate $\int_0^1 f(x)$ and look at how the error changes.

In [75]:
import numpy as np
import matplotlib.pyplot as plt

def grid(xl, xu, N, D):
    """
    Create a grid of N evenly spaced points in D-dimensional space
    xl: lower bound of x
    xu: upper bound of x
    N: number of points per dimension
    D: number of dimensions
    """
    xr = np.linspace(xl, xu, N)
    g = np.zeros((N**D, D))
    for n in range(N**D):
        index = np.unravel_index(n, tuple([N]*D))
        g[n] = [xr[i] for i in index]
        
    return g

def f(x):
    return (1 - (np.sum(np.square(x)) / x.size))

def riemann(N, D):
    # riemann integration
    x = grid(0.0, 1.0, N, D)
    dx = 1.0 / (N**D)
    F_r = np.sum(np.apply_along_axis(f, 1, x) * dx)
    return F_r

def monte_carlo(N, D):
    # monte carlo integration
    x = np.random.rand(N**D, D)
    F_mc = np.sum(np.apply_along_axis(f, 1, x)) / (N**D)
    return F_mc

D = 1
N = np.logspace(1, 3, num=10, dtype=int)

F_r = np.zeros(10)
for i,n in enumerate(N):
    F_r[i] = riemann(n, D)
    
# error in riemann estimate
e_r = np.abs(F_r - 2.0/3.0)
print(e_r)

[ 0.01851852  0.01111111  0.00641026  0.0037037   0.00219298  0.00130208
  0.00077882  0.00046555  0.00027871  0.00016683]


Below we plot the error in Riemann estimate with respect to number of sample points. As it is clear from the figure, this error varies with $O(N^{-1})$.

In [76]:
plt.plot(N, e_r)
plt.plot(N, (e_r[0]*N[0])/N)
plt.legend(['Error in Riemann estimate', '1/N'])
plt.show()

In [44]:
repeats = 1000
e_mc = np.zeros(10)
for i,n in enumerate(N):
    for r in range(repeats):
        e_mc[i] += np.abs(monte_carlo(n, D) - 2.0/3.0)

e_mc /= repeats
print(e_mc)

[ 0.0740192   0.06172955  0.04757969  0.03471028  0.02688691  0.02152748
  0.01563089  0.01264551  0.00963057  0.00727757]


Now let us look at the error in the Monte Carlo estimate. Again, as we have seen above, the error in Monte Carlo estimate varies with $O(n^{-1/2})$.

In [52]:
plt.plot(N, e_mc)
plt.plot(N, (e_mc[0]*np.sqrt(N[0]))/np.sqrt(N))
plt.legend(['Error in Monte Carlo estimate', r"$\frac{1}{N^{-1/2}}$"])
plt.show()

Then, why would anyone use Monte Carlo integration? Obviously, Riemann estimate decreases much more quickly as we increase the number of sample points. What is the deal with the error rate in Monte Carlo integration being independent of the number of dimensions? Do the Monte Carlo and Riemann errors change differently as we increase the number of dimensions? Let us look into that.

First, we need to understand how the Riemann error changes if we go to a higher dimensional space. Let $N$ denote the number of points along each axis; hence, if we have $D$ dimensions, we have $N^D$ sample points. In a D-dimensional space, does the error change as $O(1/N)$ or $O(1/(N^D))$? Let us try $D=2$.

In [115]:
D = 2
N = np.array([5, 10, 20 , 50, 100])

F_rd = np.zeros(5)
for i,n in enumerate(N):
    F_rd[i] = riemann(n, D)
    
e_rd = np.abs(F_rd - 2.0/3.0)

In [119]:
plt.plot(N, e_rd)
plt.plot(N, (e_rd[0]*N[0])/(N))
plt.plot(N, (e_rd[0]*(N[0]**D))/(N**D))
plt.legend(['Error in Riemann estimate', '1/N', '1/N^D'])
plt.show()

It looks more like $O(1/N)$. Let us look at $D=3$ too.

In [123]:
D = 3
N = np.array([3, 6, 12 , 25, 50])

F_rd = np.zeros(5)
for i,n in enumerate(N):
    F_rd[i] = riemann(n, D)
    
e_rd = np.abs(F_rd - 2.0/3.0)

In [124]:
plt.plot(N, e_rd)
plt.plot(N, (e_rd[0]*N[0])/(N))
plt.plot(N, (e_rd[0]*(N[0]**D))/(N**D))
plt.legend(['Error in Riemann estimate', '1/N', '1/N^D'])
plt.show()

Again it looks more like $O(1/N)$. Another way to look at that is to see how the error changes as we change the number of dimensions.

In [125]:
D = [1,2,3,4,5]
N = 10

F_rn = np.zeros(5)
for i,d in enumerate(D):
    F_rn[i] = riemann(N, d)
    
e_rn = np.abs(F_rn - 2.0/3.0)

In [127]:
plt.plot(D, e_rn)
plt.show()

As it is clear, if we keep the number of points per dimension constant, the error stays constant. In other words, Riemann error behaves as $O(1/N)$. That also means that if we keep the number of sample (grid) points constant, Riemann error should increase.

In [114]:
D = np.array([1, 2, 4, 8, 16])
# we need to keep the number of samples constant across dimensions
N = np.array([65536, 256, 16, 4, 2])
F_rc = np.zeros(5)
for i,(n,d) in enumerate(zip(N,D)):
    F_rc[i] = riemann(n, d)
    
# error in riemann estimate
e_rc = np.abs(F_rc - (2.0 / 3.0))
print(e_rc)

In [133]:
plt.plot(D, e_rc)
plt.plot(D, (e_rc[0]*N[0])/N)
plt.legend(['Error in Riemann estimate', '1/N'])
plt.show()

Though the increase seems to be larger than $O(1/N)$.

Nevertheless, for us the important point is to compare how the Monte Carlo error changes to Riemann error. Let us look at how the Monte Carlo error changes with respect to $N$ for $D=2$. Remember that $N$ is the number of points per dimension.

In [136]:
repeats = 100
D = 2
N = np.array([5, 10, 20 , 50, 100])
e_mcd2 = np.zeros(5)
for i,n in enumerate(N):
    for r in range(repeats):
        e_mcd2[i] += np.abs(monte_carlo(n, D) - (2.0/3.0))

e_mcd2 /= repeats
print(e_mcd2)

[ 0.03473115  0.01715977  0.00924366  0.00337691  0.00177935]


In [139]:
plt.plot(N, e_mcd2)
plt.plot(N, (e_mcd2[0]*(N[0]))/N)
plt.legend(['Error in Monte Carlo estimate', r"$\frac{1}{\sqrt{N^D}}$"])
plt.show()

Ahah! Note that the Monte Carlo error changes as $O(1/\sqrt{N^D})$ which is $O(1/N)$ in this case. In other words, Monte Carlo error depends on the total number of sample points, not on the number of points per dimension (as Riemann error does). For example, if we keep $N$ constant as we increase the number of dimensions, the Monte Carlo error should decrease. Remember, Riemann error stays the same as long as $N$ is constant.

In [140]:
repeats = 100
D = np.array([1, 2, 3, 4, 5])
# we need to keep the number of samples constant across dimensions
N = 10
e_mcd = np.zeros(5)
for i,d in enumerate(D):
    for r in range(repeats):
        e_mcd[i] += np.abs(monte_carlo(N, d) - (2.0/3.0))

e_mcd /= repeats
print(e_mcd)

[ 0.08284523  0.01660181  0.00478395  0.00116964  0.000303  ]


In [145]:
plt.plot(D, e_mcd)
D = np.array([1, 2, 3, 4, 5])
plt.plot(D, (e_mcd[0]*np.sqrt(N))/np.sqrt(N**D))
plt.legend(['Error in Monte Carlo estimate', r"$\frac{1}{\sqrt{N^D}}$"])
plt.show()

Note that how the Monte Carlo error changes as $\frac{1}{\sqrt{N^D}}$. Therefore, one only needs a constant number of  sample points to achieve a constant error rate. In contrast, for Riemann integration, we need to keep the number of points per dimension constant, which means an exponentially increasing number of sample points as $D$ increases.

To recap, for Monte Carlo integration, the error varies with $O(1/\sqrt{N^D})$ where $N$ is the number of points per dimension. In contrast, for Riemann integration, the error varies with $O(1/N)$. Therefore, if we keep the total number of sample points constant, Riemann estimate will get worse as $D$ increases, while Monte Carlo error should stay the same. 